In [1]:
import pandas as pd 
import numpy as np 
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import lightgbm as lgb
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test_QkPvNLx.csv')
calender = pd.read_csv('calender_1.csv')

all_df = train_df.append(test_df)

all_df = all_df.merge(calender, how= 'left', on = 'Day_No')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [2]:
all_df['User_Traffic'] = all_df.groupby('Course_ID')['User_Traffic'].transform(lambda x : x.rolling(7).mean())
all_df['User_Traffic'] = np.log1p(all_df['User_Traffic'])
pred_user_traffic = pd.Series(index = all_df.index, data = None)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
train_period = (0,60)
pred_period = (61,120)

## Some manual intervention required to make for OOT. 

while(True):
    if pred_period[0] >= 942: continue
    day_train_start = train_period[0]
    day_train_end = train_period[1]
    day_valid_start = pred_period[0]
    day_valid_end = pred_period[1]
    
    tr = all_df.query(f"(Day_No <= {day_train_end}) & (Day_No >= {day_train_start})")
    pred = all_df.query(f"(Day_No <= {day_valid_end}) & (Day_No >= {day_valid_start})")
    
    dtrain = lgb.Dataset(tr[trainCols], tr['User_Traffic'], free_raw_data=False, silent=True)
    dvalid = lgb.Dataset(pred[trainCols], pred['User_Traffic'], free_raw_data=False, silent=True)
    
    lgb_params = {
                'objective': 'regression',
                'boosting_type': 'gbdt',
                'learning_rate': .1,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'num_leaves': 31,
                'max_depth': -1,
                'seed': 13,
                'n_jobs': 8,
                'min_split_gain': .00001,
                'reg_alpha': .00001,
                'reg_lambda': .00001,
                'metric': 'rmse'
                }

    # Fit the model
    hist = lgb.train(
                params=lgb_params, 
                train_set=dtrain,
                valid_sets= [dtrain],
                num_boost_round=3000,
                categorical_feature=catCols,
                early_stopping_rounds=100,
                verbose_eval=0)
    
    prediction = hist.predict(pred[trainCols])
    
    try:
        print(f"RMSE : {mean_squared_error(pred['User_Traffic'], prediction)}")
    except:
        print(f"Done Till {pred_period[1]}")
        print("Entered OOT Phase")
        pred_user_traffic.iloc[pred.index] = prediction
        break
    
    pred_user_traffic.iloc[pred.index] = prediction
    
    print(f"Done Till {pred_period[1]}")
    train_period = (0, train_period[1] + 60)
    pred_period = (train_period[1] + 1, train_period[1] + 60)

categorical_feature in Dataset is overridden.
New categorical_feature is ['Course_Domain', 'Course_ID', 'Course_Type']


Done Till 942
Entered OOT Phase


In [11]:
pred_user_traffic

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
            ...   
548082    8.957769
548083    8.964530
548084    9.057988
548085    9.111396
548086    9.098207
Length: 548087, dtype: float64

In [12]:
temp = all_df[['ID', 'Course_ID','User_Traffic']]

In [13]:
temp['User_Traffic_rolled_predicted'] = pred_user_traffic


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
temp_cid = temp.query('Course_ID == 544')

In [15]:
temp.to_csv('user_traffic_pred.csv', index = False)